In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(726, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(726,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)
        y_batch = np.eye(2)[self.y[batch_indices]]
        return X_batch, y_batch
    
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")
        
class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        """
        Early stopping within the same epoch based on a monitored metric.
        Args:
            monitor: Metric to monitor ('loss', 'accuracy', etc.).
            threshold: Threshold value for stopping (e.g., loss < 0.1).
            patience: Number of batches to wait for improvement before stopping.
        """
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0 

batch_early_stopping_callback = BatchEarlyStopping(
    monitor='loss',     
    threshold=0.1,       
    patience=2          
)

def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  

    sequence_input = Input(shape=(seq_length, height, width, channels))
    
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128)(flattened_features)
    lstm_out = Dropout(0.5)(lstm_out)

    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.5)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

indices = np.arange(726)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', train_indices, batch_size=5)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', val_indices, batch_size=5)

checkpoint_dir = './Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',  
    save_best_only=False,
    save_weights_only=False,  
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2: 
        return lr
    return lr * 0.9 

lr_callback = LearningRateScheduler(scheduler)

step_timer_callback = StepTimerCallback()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1  
)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,993,410 (110.60 MB)

 Trainable params: 25,764,546 (98.28 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/10
Step 1/116 - Loss: 0.7347, Accuracy: 0.6000, Time: 72.43 seconds
  1/116 ━━━━━━━━━━━━━━━━━━━━ 2:22:13 74s/step - accuracy: 0.6000 - loss: 0.7347Step 2/116 - Loss: 0.6449, Accuracy: 0.6000, Time: 3.32 seconds
  2/116 ━━━━━━━━━━━━━━━━━━━━ 6:37 3s/step - accuracy: 0.6000 - loss: 0.6898    Step 3/116 - Loss: 0.7166, Accuracy: 0.5333, Time: 1.85 seconds
  3/116 ━━━━━━━━━━━━━━━━━━━━ 5:01 3s/step - accuracy: 0.5778 - loss: 0.6987Step 4/116 - Loss: 0.8504, Accuracy: 0.5000, Time: 1.77 seconds
  4/116 ━━━━━━━━━━━━━━━━━━━━ 4:25 2s/step - accuracy: 0.5583 - loss: 0.7367Step 5/116 - Loss: 1.0306, Accuracy: 0.4800, Time: 1.89 seconds
  5/116 ━━━━━━━━━━━━━━━━━━━━ 4:09 2s/step - accuracy: 0.5427 - loss: 0.7954Step 6/116 - Loss: 1.1032, Accuracy: 0.4667, Time: 1.80 seconds
  6/116 ━━━━━━━━━━━━━━━━━━━━ 3:57 2s/step - accuracy: 0.5300 - loss: 0.8467Step 7/116 - Loss: 1.0210, Accuracy: 0.4857, Time: 1.80 seconds
  7/116 ━━━━━━━━━━━━━━━━━━━━ 3:48 2s/step - accuracy: 0.

In [2]:
import os
import time
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, TimeDistributed, Flatten, LSTM, Dropout, Dense
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(726, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(726,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  
        y_batch = np.eye(2)[self.y[batch_indices]]  
        return X_batch, y_batch

class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None and current_value < self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                self.model.stop_training = True
        else:
            self.wait = 0

batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  # Freeze the pre-trained model

    sequence_input = Input(shape=(seq_length, height, width, channels))
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128)(flattened_features)
    lstm_out = Dropout(0.5)(lstm_out)

    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.5)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

indices = np.arange(726)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', train_indices, batch_size=5)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', val_indices, batch_size=5)

checkpoint_dir = './Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    return lr * 0.9

lr_callback = LearningRateScheduler(scheduler)

latest_checkpoint = './Checkpoints/model_epoch_06_val_loss_0.68.keras'  # Update to your latest checkpoint

cnn_lstm_model = load_model(latest_checkpoint)

cnn_lstm_model.summary()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10, 
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, StepTimerCallback(), batch_early_stopping_callback],
    verbose=1
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,522,504 (307.17 MB)

 Trainable params: 25,764,546 (98.28 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

 Optimizer params: 51,529,094 (196.57 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/10
Step 1/116 - Loss: 0.6182, Accuracy: 0.6000, Time: 76.05 seconds
  1/116 ━━━━━━━━━━━━━━━━━━━━ 2:30:21 78s/step - accuracy: 0.6000 - loss: 0.6182Step 2/116 - Loss: 0.6920, Accuracy: 0.5000, Time: 2.32 seconds
  2/116 ━━━━━━━━━━━━━━━━━━━━ 4:42 2s/step - accuracy: 0.5500 - loss: 0.6551    Step 3/116 - Loss: 0.7408, Accuracy: 0.4000, Time: 1.84 seconds
  3/116 ━━━━━━━━━━━━━━━━━━━━ 4:03 2s/step - accuracy: 0.5000 - loss: 0.6837Step 4/116 - Loss: 0.7307, Accuracy: 0.4000, Time: 1.92 seconds
  4/116 ━━━━━━━━━━━━━━━━━━━━ 3:52 2s/step - accuracy: 0.4750 - loss: 0.6954Step 5/116 - Loss: 0.7118, Accuracy: 0.4400, Time: 1.90 seconds
  5/116 ━━━━━━━━━━━━━━━━━━━━ 3:45 2s/step - accuracy: 0.4680 - loss: 0.6987Step 6/116 - Loss: 0.7294, Accuracy: 0.4000, Time: 1.96 seconds
  6/116 ━━━━━━━━━━━━━━━━━━━━ 3:42 2s/step - accuracy: 0.4567 - loss: 0.7038Step 7/116 - Loss: 0.7134, Accuracy: 0.4571, Time: 1.84 seconds
  7/116 ━━━━━━━━━━━━━━━━━━━━ 3:36 2s/step - accuracy: 0.

In [3]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8197 - loss: 0.6678
Validation Loss: 0.6673
Validation Accuracy: 0.8207


In [4]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

29/29 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.96      0.70      0.81        79
     Class 1       0.73      0.97      0.83        66

    accuracy                           0.82       145
   macro avg       0.85      0.83      0.82       145
weighted avg       0.86      0.82      0.82       145

Confusion Matrix:
[[55 24]
 [ 2 64]]


In [5]:
saved_model_path = './saved_model/cnn_lstm_model_try2.h5'

cnn_lstm_model.save(saved_model_path)

print(f"Model saved to {saved_model_path}")

Model saved to ./saved_model/cnn_lstm_model_try2.h5


In [26]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = './Checkpoints/model_epoch_10_val_loss_0.67.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_7.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


Model loaded successfully!
Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9972225427627563
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step
Raw Predictions: [[0.4947524  0.50524753]
 [0.5044213  0.49557874]]
Predicted Class Probabilities: [0.4947524  0.50524753]
Predicted Class: [1 0]


In [1]:
import os
import time
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, TimeDistributed, Flatten, LSTM, Dropout, Dense
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(726, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(726,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  
        y_batch = np.eye(2)[self.y[batch_indices]]  
        return X_batch, y_batch

class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None and current_value < self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                self.model.stop_training = True
        else:
            self.wait = 0

batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  # Freeze the pre-trained model

    sequence_input = Input(shape=(seq_length, height, width, channels))
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128)(flattened_features)
    lstm_out = Dropout(0.5)(lstm_out)

    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.5)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

indices = np.arange(726)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', train_indices, batch_size=5)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', val_indices, batch_size=5)

checkpoint_dir = './MoreCheckpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    return lr * 0.9

lr_callback = LearningRateScheduler(scheduler)

latest_checkpoint = './Checkpoints/model_epoch_10_val_loss_0.67.keras'  # Update to your latest checkpoint

cnn_lstm_model = load_model(latest_checkpoint)

cnn_lstm_model.summary()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10, 
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, StepTimerCallback(), batch_early_stopping_callback],
    verbose=1
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,522,504 (307.17 MB)

 Trainable params: 25,764,546 (98.28 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

 Optimizer params: 51,529,094 (196.57 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/10
Step 1/116 - Loss: 0.7683, Accuracy: 0.4000, Time: 79.13 seconds
  1/116 ━━━━━━━━━━━━━━━━━━━━ 2:47:03 87s/step - accuracy: 0.4000 - loss: 0.7683Step 2/116 - Loss: 0.7259, Accuracy: 0.4000, Time: 3.77 seconds
  2/116 ━━━━━━━━━━━━━━━━━━━━ 7:41 4s/step - accuracy: 0.4000 - loss: 0.7471    Step 3/116 - Loss: 0.7148, Accuracy: 0.4667, Time: 1.82 seconds
  3/116 ━━━━━━━━━━━━━━━━━━━━ 5:31 3s/step - accuracy: 0.4222 - loss: 0.7363Step 4/116 - Loss: 0.7151, Accuracy: 0.5000, Time: 1.84 seconds
  4/116 ━━━━━━━━━━━━━━━━━━━━ 4:47 3s/step - accuracy: 0.4417 - loss: 0.7310Step 5/116 - Loss: 0.7154, Accuracy: 0.4800, Time: 1.96 seconds
  5/116 ━━━━━━━━━━━━━━━━━━━━ 4:28 2s/step - accuracy: 0.4493 - loss: 0.7279Step 6/116 - Loss: 0.7077, Accuracy: 0.5333, Time: 1.87 seconds
  6/116 ━━━━━━━━━━━━━━━━━━━━ 4:14 2s/step - accuracy: 0.4633 - loss: 0.7245Step 7/116 - Loss: 0.7158, Accuracy: 0.5143, Time: 1.78 seconds
  7/116 ━━━━━━━━━━━━━━━━━━━━ 4:02 2s/step - accuracy: 0.

In [2]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.7978 - loss: 0.6115
Validation Loss: 0.6047
Validation Accuracy: 0.8276


In [3]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

29/29 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.85      0.84      0.84        79
     Class 1       0.81      0.82      0.81        66

    accuracy                           0.83       145
   macro avg       0.83      0.83      0.83       145
weighted avg       0.83      0.83      0.83       145

Confusion Matrix:
[[66 13]
 [12 54]]


In [5]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\MoreCheckpoints\model_epoch_10_val_loss_0.60.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_7.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


Model loaded successfully!
Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9972225427627563
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step
Raw Predictions: [[0.51817274 0.48182732]
 [0.5399267  0.46007326]]
Predicted Class Probabilities: [0.51817274 0.48182732]
Predicted Class: [0 0]


In [6]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

In [8]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\MoreCheckpoints\model_epoch_10_val_loss_0.60.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


Model loaded successfully!
Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9970716238021851
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step
Raw Predictions: [[0.42865348 0.5713465 ]
 [0.42022324 0.57977676]]
Predicted Class Probabilities: [0.42865348 0.5713465 ]
Predicted Class: [1 1]


In [2]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\MoreCheckpoints\model_epoch_10_val_loss_0.60.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_8.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


Model loaded successfully!
Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9958799481391907
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step
Raw Predictions: [[0.5610093  0.43899068]
 [0.5638585  0.43614152]]
Predicted Class Probabilities: [0.5610093  0.43899068]
Predicted Class: [0 0]


In [2]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\MoreCheckpoints\model_epoch_10_val_loss_0.60.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_7.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


Model loaded successfully!
Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9972225427627563
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step
Raw Predictions: [[0.51817274 0.48182732]
 [0.5399267  0.46007326]]
Predicted Class Probabilities: [0.51817274 0.48182732]
Predicted Class: [0 0]
